# (1) 종속 패키지 설치

In [ ]:
# 종속 패키지(dependencies) 설치(pytorch, detectron2)
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__

!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

# 인스톨이 완료되면 Colab Runtime을 다시 시작해주세요. 

# (2) 유틸 Import

In [1]:
# 기본 설정
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import itertools

# detectron2 logger 설정
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# 자주 사용하는 라이브러리 임폴트
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# 자주 사용하는 detectron2 유틸 임폴트 
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog, MetadataCatalog

# (3) github에서 TRAIN, VAL data들 가져오기

In [ ]:
!git clone https://github.com/sosobam/AI_Jump_up.git

# (4) detectron2에서 데이터셋 정보를 로드하는 형식에 맞춰 함수를 작성

In [3]:
# 이미지 경로에서 Data set 뽑는 함수
def get_PTN_dicts(img_dir):
    json_file = os.path.join(img_dir, "annotations.json")
    with open(json_file) as f:
        imgs_anns = json.load(f)

    # 최종 결과물
    dataset_dicts = []
    
    idx = 0
    # 이미지들을 annotation한 json에서 이미지별로 하나씩 데이터 뽑아서 record에 저장(key : 이미지file.jpg, val : seg데이터)
    for key, val in imgs_anns.items():
        record = {} # 각 이미지별 정보 담을 dataset 포맷
        
        filename = os.path.join(img_dir, key) # 이미지file 경로
        height, width = cv2.imread(filename).shape[:2] # 이미지file 경로로 cv2로 이미지 열어서 height, width 추출
        
        # 이미지 기본정보
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
      
        annos = val["instances"]
        objs = []

        # 이미지 기본정보 중 annotation_class별로 point들(x,y,x,y,x,y,...x,y순서) 뽑아서 obj에 담고 objs에 class별로 누적 후 record["annotations"]에 최종 저장
        for anno in annos:
            points = anno["points"]
            px = []
            py = []

            # point들(x,y,x,y,x,y,...x,y순서) 뽑아서 px, py로 각각 list 생성
            for i in range(0,int(len(points)/2)):
              px = px + [points[i*2]] # 짝수
              py = px + [points[i*2+1]] # 홀수

            # px, py list를 poly에 (x,y)형태로 묶어서 다시 list화
            poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            poly = list(itertools.chain.from_iterable(poly))

            # obj dict에 (x,y)list를 토대로 Box만들고 list 등록
            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": 0,
                "iscrowd": 0
            }
            objs.append(obj) # obj를 objs에 누적
        record["annotations"] = objs # objs를 record["annotations"]에 등록
        dataset_dicts.append(record) # record를 dataset에 누적
    return dataset_dicts

# (5) train, val 이미지들을 metadata set에 등록

In [ ]:
for d in ["TRAIN", "VAL"]:
    DatasetCatalog.register("PTN_" + d, lambda d=d: get_PTN_dicts("AI_Jump_up/"+d))
    MetadataCatalog.get("PTN_" + d).set(thing_classes=["PTN"])
PTN_metadata = MetadataCatalog.get("PTN_TRAIN")

# (6) 데이터가 제대로 로드되었는지 확인

In [ ]:
dataset_dicts = get_PTN_dicts("AI_Jump_up/TRAIN")
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=PTN_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

# (7) 학습하기

## (7.1) Instance Segmentation 학습

자 이제 학습을 진행할 단계입니다. COCO 데이터셋으로 학습된 R50-FPN Mask R-CNN 모델을 불러와서 풍선 데이터셋으로 fine-tune해 봅시다.
Colab의 K80 GPU를 기준으로, 300 iterations 학습시키는데 대략 6분정도의 시간이 소요됩니다(P100 GPU의 경우 2분 가량 소요됩니다).

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("PTN_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 100    # 300 iterations 정도면 충분합니다. 더 오랜 시간도 시도해보세요.
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # 풍선 데이터셋과 같이 작은 데이터셋에서는 이정도면 적당합니다.
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # 클래스는 "풍선" 클래스 하나 뿐입니다.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

## (7.2) 학습 커브 확인

In [ ]:
# tensorboard를 사용해서 학습 커브를 살펴봅니다.
%load_ext tensorboard
%tensorboard --logdir output

## (7.3) 학습한 모델 실행 및 평가하기
자 이제 풍선 데이터셋의 검증(validation) 데이터셋으로 테스트를 해볼 차례입니다. 

우선, 방금 전 학습한 모델을 불러와서 `predictor`를 생성합니다.

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("VAL", )
predictor = DefaultPredictor(cfg)

## (7.4) 샘플 검증

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_PTN_dicts("VAL")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=balloon_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])